In [4]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])

# Iliescu Exact Solution
c = 16.0*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
ue = c*h*(0.5+sym.atan(g)/sym.pi)

# ------------------------------------------ #

ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = sym.diff(ue,t)
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

# print('u_code = ' + u_code + '\n')
# print('f_code = ' + f_code)

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

In [15]:
# %load main-SUPG.py

# Load mesh and subdomains

#method = input("0: No Filter, 1: SUPG, 2: EFR; method = ")

#method = 1
#if method == 2:
#	N = input("N = ")
#nx = 500#input("h = 1/nx, let nx = ")
method = 1
timesteps = 10
def evolve_t(nx, dt, T, u_code, f_code, sigma, mu, velocity):
    t = dt - dt
    folder = "SUPG_old/"+"h"+str(nx)+"_"
    #S = 1.0  # filtering radius factor
    degree = 2
    P = 2    # polynomial degree of FE
    R = 2

    u_exact = Expression(u_code, degree = degree, t = t)

    f = Expression(f_code, degree = R, t = t+dt)

    mesh = UnitSquareMesh(nx,nx)
    h = CellSize(mesh)
    Q = FunctionSpace(mesh, "CG", P)

    u_n = Function(Q)
    u_D = Constant(0.0)


    # Set up boundary condition
    def boundary(x, on_boundary):
        return on_boundary
    bc = DirichletBC(Q, u_D, boundary)

    # Don't Modify Below This! -----------#

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_ = Function(Q)
    u_bar = Function(Q)

    # Galerkin variational problem
    # Backward Euler
    F = v*(u - u_n)*dx + dt*(mu*dot(grad(v), grad(u))*dx \
                            + v*dot(velocity, grad(u))*dx \
                            + sigma*v*u*dx \
                            - f*v*dx)

    methodname = "SUPG"
    Lt = -mu*div(grad(u)) + dot(velocity, grad(u)) + (sigma+1.0/dt)*u 
    ft = u_/dt + f			
    r = ft - Lt
    vnorm = sqrt(dot(velocity, velocity))
    F += (h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx
    #F += (h*m.sqrt(2))*dot(velocity, grad(v))*r*dx
    #F -= (h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)


    num_steps = int(round(T / dt, 0))+1


    out_file_ubar = File(folder+"u_"+methodname+".pvd")      # filtered solution
    out_file_ubar << (u_, 0.0)
    for n in range(num_steps-1):
        if method == 1:
            u_bar.rename('SUPG','SUPG')
        else:
            u_bar.rename('No Filter','No Filter')
        b = assemble(L)
        bc.apply(b)
        solve(A1, u_bar.vector(), b, "gmres")

        if n % timesteps == 0:
            out_file_ubar << (u_bar, float(t+dt))

        # Update previous solution and source term
        u_n.assign(u_bar)
        progress.update(t / T)
        # Update current time
        t += dt
        f.t += dt
        
    L2, H1 = compute_errors(u_exact, u_bar, t, mesh)
    maxval, minval = compute_extrema(u_bar, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)
    print(t)

In [11]:
dt = 0.01
T = 0.01
for nx in [25, 50, 100]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

25 , 0.0120034836072 , 0.319719675743 , 0.0608865655571 , -0.038142365774
0.02
50 , 0.0115653869848 , 0.381018237931 , 0.0554915496446 , -0.0421634175197
0.02
100 , 0.0140219841006 , 0.379431384053 , 0.0258666866316 , -0.0417282931719
0.02


In [16]:
dt = 0.01
T = 0.5
for nx in [25, 50, 100]:
    evolve_t(nx, dt, T, u_code, f_code, sigma, mu, b)

25 , 0.0919493287391 , 4.68308590072 , 0.884478657627 , -0.783695811531
0.5
50 , 0.0984099999249 , 5.68493352026 , 0.922726526727 , -0.913381275389
0.5
100 , 0.089060669443 , 5.24898332373 , 0.623429645752 , -0.621720651726
0.5
